# Transactions analysis

In [ ]:
import pandas as pd
import ast


: 

In [3]:
file_path = "/Users/apeksha/Desktop/Xtillion_Venmito/Venmito-apekshamalik/output_data/transactions_(linked).csv"
transactions_df = pd.read_csv(file_path)

In [4]:
display(transactions_df.head())

,transaction_id,phone,store,items,person_id
0,1000,233-159-4158,Trader Tales,"['Krafty Cheddar (x1)', 'Popsi (x1)', 'Oreoz (...",854
1,1001,725-427-2794,PetPals Mart,['Colgatex (x1)'],341
2,1002,659-190-9378,Trader Tales,"['Colgatex (x1)', 'Dovee (x1)', 'Flixnet (x1)']",15
3,1003,334-436-6254,Urban Outfitters Loft,"['Krafty Cheddar (x2)', 'RedCow (x1)', 'Dovee ...",371
4,1004,615-102-4849,PetPals Mart,"['Popsi (x1)', 'Flixnet (x1)']",480


In [5]:
print(transactions_df["items"].head(10))  # See the first 10 values
print(type(transactions_df["items"].iloc[0]))  # Check the type of the first row

0    ['Krafty Cheddar (x1)', 'Popsi (x1)', 'Oreoz (...
1                                    ['Colgatex (x1)']
2      ['Colgatex (x1)', 'Dovee (x1)', 'Flixnet (x1)']
3    ['Krafty Cheddar (x2)', 'RedCow (x1)', 'Dovee ...
4                       ['Popsi (x1)', 'Flixnet (x1)']
5                      ['Popsi (x1)', 'KittyKat (x2)']
6                    ['Flixnet (x1)', 'Colgatex (x2)']
7       ['KittyKat (x2)', 'RedCow (x1)', 'Dovee (x3)']
8    ['Colgatex (x1)', 'RedCow (x1)', 'KittyKat (x1)']
9                              ['Krafty Cheddar (x1)']
Name: items, dtype: object
<class 'str'>


In [6]:
transactions_df["items"] = transactions_df["items"].apply(ast.literal_eval)

In [7]:
transactions_df = transactions_df.explode("items")

In [8]:
print(transactions_df.head(10))

   transaction_id         phone                  store                items  \
0            1000  233-159-4158           Trader Tales  Krafty Cheddar (x1)   
0            1000  233-159-4158           Trader Tales           Popsi (x1)   
0            1000  233-159-4158           Trader Tales           Oreoz (x2)   
1            1001  725-427-2794           PetPals Mart        Colgatex (x1)   
2            1002  659-190-9378           Trader Tales        Colgatex (x1)   
2            1002  659-190-9378           Trader Tales           Dovee (x1)   
2            1002  659-190-9378           Trader Tales         Flixnet (x1)   
3            1003  334-436-6254  Urban Outfitters Loft  Krafty Cheddar (x2)   
3            1003  334-436-6254  Urban Outfitters Loft          RedCow (x1)   
3            1003  334-436-6254  Urban Outfitters Loft           Dovee (x2)   

   person_id  
0        854  
0        854  
0        854  
1        341  
2         15  
2         15  
2         15  
3        3

In [9]:
best_sellers = transactions_df["items"].value_counts().reset_index()
best_sellers.columns = ['Item', 'Count']


# Reformat to extract and remove quantity amounts

In [10]:
transactions_df["item_clean"] = transactions_df["items"].str.extract(r'(.+) \(x\d+\)')
transactions_df["quantity"] = transactions_df["items"].str.extract(r'x(\d+)').astype(float)

# Best sellers dataframe cleaned and formatted in descending order

In [11]:
best_sellers = transactions_df.groupby("item_clean")["quantity"].sum().reset_index()
best_sellers = best_sellers.sort_values(by="quantity", ascending=False)

# Best 10 selling items

In [12]:
display(best_sellers.head(10))

,item_clean,quantity
2,Dovee,96.0
9,RedCow,66.0
1,Colgatex,58.0
8,Popsi,58.0
3,Flixnet,50.0
5,KittyKat,46.0
6,Krafty Cheddar,30.0
7,Oreoz,28.0
4,GatorBoost,17.0
0,Coca-Splash,2.0


# Format for best 10 and handling NaN

In [13]:
transactions_df["item_clean"] = transactions_df["items"].str.extract(r'(.+?) \(x\d+\)')
transactions_df["quantity"] = transactions_df["items"].str.extract(r'x(\d+)').astype(float)

transactions_df["quantity"].fillna(1, inplace=True)

/var/folders/y4/dyhvw0wj3ll75465hnk5bhrh0000gn/T/ipykernel_11819/1222971556.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  transactions_df["quantity"].fillna(1, inplace=True)


In [14]:
store_sales = transactions_df.groupby("store")["quantity"].sum().reset_index().sort_values(by="quantity", ascending=False)

# Stores with the highest sales display

In [15]:
display(store_sales.head(10))

,store,quantity
5,Urban Outfitters Loft,134.0
4,Trader Tales,109.0
1,PetPals Mart,99.0
0,BestChoice Buy,57.0
2,SparkMart,44.0
3,Targeted Treasures,8.0
